In [78]:
import os
import numpy as np
import cv2
from joblib import dump, Parallel, delayed

def get_image_paths(maxthings):
    imagepaths = []
    mid_path = os.path.join(os.getcwd(),"images")
    if(maxthings):
        for img_name in os.listdir(mid_path)[:maxthings]:
            imagepaths.append(os.path.join(mid_path,img_name))
    else:
        for img_name in os.listdir(mid_path):
            imagepaths.append(os.path.join(mid_path,img_name))
        
    return imagepaths

def getMeans(imgp):
    img = cv2.imread(imgp)
    halfX = len(img[0])//2
    halfY = len(img)//2
    q1 = np.mean(img[:halfY][:,halfX:,:], axis=(0,1))
    q2 = np.mean(img[:halfY][:,:halfX,:], axis=(0,1))
    q3 = np.mean(img[halfY:][:,:halfX,:], axis=(0,1))
    q4 = np.mean(img[halfY:][:,halfX:,:], axis=(0,1))
    means = [q1,q2,q3,q4]
    return [[np.mean(means, axis=0),means],imgp]

c_map = Parallel(n_jobs=3)(delayed(getMeans)(imgp) for imgp in get_image_paths(None))
dump(c_map,"colormap.joblib")

['colormap.joblib']